In [1]:
! pip install dataset transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.17
    Uninstalling SQLAlchemy-2.0.17:
      Successfully uninstalled SQLAlchemy-2.0.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


In [3]:
from datasets import load_dataset
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
import numpy as np

In [401]:
checkpoint = "bert-base-uncased"

tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=TFAutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [402]:
tokenizer(['i like meet,i hate food'])

{'input_ids': [[101, 1045, 2066, 3113, 1010, 1045, 5223, 2833, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [453]:
dataset = load_dataset("csv", data_files="/kaggle/input/email-spam-detection-dataset-classification/spam.csv",encoding='ISO-8859-1',split='train')                    

In [454]:
dataset

Dataset({
    features: ['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],
    num_rows: 5572
})

In [404]:
dataset.features

{'v1': Value(dtype='string', id=None),
 'v2': Value(dtype='string', id=None),
 'Unnamed: 2': Value(dtype='string', id=None),
 'Unnamed: 3': Value(dtype='string', id=None),
 'Unnamed: 4': Value(dtype='string', id=None)}

In [405]:
dataset=dataset.rename_columns({'v1':'label','v2':'email'})

In [406]:
dataset=dataset.remove_columns(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

In [407]:
dataset[:10]

{'label': ['ham',
  'ham',
  'spam',
  'ham',
  'ham',
  'spam',
  'ham',
  'ham',
  'spam',
  'spam'],
 'email': ['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  'Ok lar... Joking wif u oni...',
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  'U dun say so early hor... U c already then say...',
  "Nah I don't think he goes to usf, he lives around here though",
  "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",
  'Even my brother is not like to speak with me. They treat me like aids patent.',
  "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
  'WINNER!! As a valued network customer you have

In [408]:
dataset

Dataset({
    features: ['label', 'email'],
    num_rows: 5572
})

In [409]:
dataset=dataset.map(lambda x:{'label':1 if x['label']=='ham' else 0} )
dataset=dataset.map(lambda x:tokenizer(x['email'],truncation=True))
dataset[0:10]


  0%|          | 0/5572 [00:00<?, ?ex/s]

  0%|          | 0/5572 [00:00<?, ?ex/s]

{'label': [1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
 'email': ['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  'Ok lar... Joking wif u oni...',
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  'U dun say so early hor... U c already then say...',
  "Nah I don't think he goes to usf, he lives around here though",
  "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",
  'Even my brother is not like to speak with me. They treat me like aids patent.',
  "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
  'WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 0

In [410]:
data_colletor=DataCollatorWithPadding(tokenizer=tokenizer,return_tensors='tf')


In [411]:
dataset=dataset.train_test_split(shuffle=True)
dataset['validation']=dataset['train'].train_test_split(shuffle=True)['test']
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'email', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4179
    })
    test: Dataset({
        features: ['label', 'email', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1393
    })
    validation: Dataset({
        features: ['label', 'email', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1045
    })
})

In [412]:
train_data=dataset['train']
validation_data=dataset['validation']
test_data=dataset['test']

In [413]:
train_data=train_data.to_tf_dataset(columns=['input_ids', 'token_type_ids', 'attention_mask'],
    batch_size=8,
    shuffle=True,
    collate_fn=data_colletor,
    label_cols=['label'])
validation_data=validation_data.to_tf_dataset(columns=['input_ids', 'token_type_ids', 'attention_mask'],
    batch_size=8,
    shuffle=False,
    collate_fn=data_colletor,
    label_cols=['label'])
test_data=test_data.to_tf_dataset(columns=['input_ids', 'token_type_ids', 'attention_mask'],
    batch_size=8,
    shuffle=False,
    collate_fn=data_colletor,
    label_cols=['label'])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [414]:
batch_size = 8
num_epochs = 2
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(train_data) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = Adam(learning_rate=lr_scheduler)
model.compile(
    optimizer=opt,
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    train_data,epochs=1,validation_data=validation_data
)

522/522 [==============================] - 150s 200ms/step - loss: 0.0770 - accuracy: 0.9765 - val_loss: 0.0152 - val_accuracy: 0.9971


In [415]:
def compute_accuracy(model,test_data):
    prediction=model.predict(test_data)['logits']
    prediction=np.argmax(prediction,axis=-1)
    accuracy=np.sum(prediction==dataset['test']['label'])/len(prediction)
    return f'The Acuuracy Is: {accuracy}'

In [416]:
compute_accuracy(model,test_data)

175/175 [==============================] - 12s 46ms/step


'The Acuuracy Is: 0.990667623833453'

In [440]:
def prediction(sample):
    encoded_sample = tokenizer.encode(sample,truncation=True)
    input_data = np.array([encoded_sample])
    prediction=model.predict(input_data)['logits']
    pred=np.argmax(prediction,axis=-1)
    label='ham'if pred[0]==1 else 'spam'
    return f'This Email Is: {label}'

In [448]:
print(dataset['test']['email'][0])
print(prediction(dataset['test']['email'][0]))

Er mw im filled tuth is aight
1/1 [==============================] - 0s 51ms/step
This Email Is: ham


In [452]:
print(dataset['test']['email'][2])
print(prediction(dataset['test']['email'][2]))

4mths half price Orange line rental & latest camera phones 4 FREE. Had your phone 11mths ? Call MobilesDirect free on 08000938767 to update now! or2stoptxt
1/1 [==============================] - 0s 55ms/step
This Email Is: spam
